In [1]:
import requests
from bs4 import BeautifulSoup
import time
import random
import pandas as pd

In [2]:
import sys
import os

In [3]:
def flushprint(www):
    sys.stdout.write('\r')
    sys.stdout.write('%s' %www)
    sys.stdout.flush()

### 城市景点类型及数量

In [4]:
attractions_url='https://www.tripadvisor.cn/Attractions-g294212-Activities-Beijing.html'

In [5]:
html=requests.get(attractions_url)
html.encoding='utf-8'
content=html.text
soup=BeautifulSoup(content,'html.parser')

In [6]:
attr_type=[]
for item in soup.find('div',class_='filter_wrap_0').find_all('a',class_='taLnk'):
    attr_type_name=item.text
    attr_type_link=item['href']
    attr_type_=[attr_type_name,attr_type_link]
    attr_type.append(attr_type_)

In [7]:
attr_type

[['景点与地标 (767)', '/Attractions-g294212-Activities-c47-Beijing.html'],
 ['自然与公园 (275)', '/Attractions-g294212-Activities-c57-Beijing.html'],
 ['博物馆 (303)', '/Attractions-g294212-Activities-c49-Beijing.html'],
 ['水上活动与游乐园 (48)', '/Attractions-g294212-Activities-c52-Beijing.html'],
 ['动物园与水族馆 (10)', '/Attractions-g294212-Activities-c48-Beijing.html'],
 ['夜生活 (79)', '/Attractions-g294212-Activities-c20-Beijing.html'],
 ['水疗与养生 (62)', '/Attractions-g294212-Activities-c40-Beijing.html'],
 ['休闲与游戏 (60)', '/Attractions-g294212-Activities-c56-Beijing.html']]

In [8]:
df_attr=pd.DataFrame(attr_type,columns=['attr-type-name','attr-type-link'])

In [9]:
df_attr.to_excel('beijing_attr_type.xls')

### 爬取某一城市所有景点以及url

In [10]:
attr_type=pd.read_excel('beijing_attr_type.xls')

In [11]:
attr_type.head()

,attr-type-name,attr-type-num,attr-type-link
0,景点与地标,767,/Attractions-g294212-Activities-c47-Beijing.html
1,自然与公园,275,/Attractions-g294212-Activities-c57-Beijing.html
2,博物馆,303,/Attractions-g294212-Activities-c49-Beijing.html
3,水上活动与游乐园,48,/Attractions-g294212-Activities-c52-Beijing.html
4,动物园与水族馆,10,/Attractions-g294212-Activities-c48-Beijing.html


In [12]:
# 翻页：一页30条数据
def page_offset(num):
    offset_range=0
    if int(num)%30==0:
        offset_range=int(num)
    else:
        offset_range=(int(num)//30+1)*30
    return offset_range

In [13]:
#城市经典页面的翻页url准备好：
attractions=[]

In [ ]:
for index in attr_type.index:
    attr_type_num=attr_type.iloc[index]['attr-type-num']
    page_range=page_offset(attr_type_num)
    for page in range(0,page_range,30):
        if page==0:
            url='https://www.tripadvisor.cn'+attr_type.iloc[index]['attr-type-link']
            try:
                tt = random.random()
                flushprint(str(url)+':sleeping'+str(tt)+'seconds')
                time.sleep(tt)

                html_=requests.get(url)
                html_.encoding='utf-8'
                content_=html_.text
                soup_=BeautifulSoup(content_,'html.parser')
                for txt in soup_.select('.listing_title  a'):
                    attr_name=txt.text
                    attr_link=txt['href']
                    attr_list=[attr_name,attr_link]
                    attractions.append(attr_list)    
            except Exception as e:
                print (url,e)
            
        else:
            attr_type_link2=attr_type.iloc[index]['attr-type-link'].replace('-Chongqing.html','')
            url='https://www.tripadvisor.cn'+attr_type_link2+'-oa'+str(page)+'-Chongqing.html#FILTERED_LIST'
            try:
                tt = random.random()
                flushprint(str(url)+':sleeping'+str(tt)+'seconds')
                time.sleep(tt)

                html_=requests.get(url)
                html_.encoding='utf-8'
                content_=html_.text
                soup_=BeautifulSoup(content_,'html.parser')
                
                for txt in soup_.select('.listing_title  a'):
                    attr_name=txt.text
                    attr_link=txt['href']
                    attr_list=[attr_name,attr_link]
                    attractions.append(attr_list)                     
            except Exception as e:
                print (url,e)

https://www.tripadvisor.cn/Attractions-g294212-Activities-c57-Beijing.html-oa270-Chongqing.html#FILTERED_LIST:sleeping0.5929789648662769secondsss

In [16]:
df_attrations=pd.DataFrame(attractions,columns=['attr-name','attr-link'])

In [19]:
df_attrations.to_excel('tianjin_attractions.xls')